In [1]:
#라이브러리 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
import IPython.display as ipd

In [2]:
#파일 읽어오기
fid = open('noisy.raw','rb')
data = np.fromfile(fid,dtype='int16',sep = "")
fid.close()

In [86]:
#샘플링 레이트 16000
sr = 16000
ipd.Audio(data,rate = sr)

In [87]:
#제로패딩 진행
new_data = np.pad(data, (0,51), 'constant', constant_values=0)

In [111]:
N = len(new_data)
WINDOW_SIZE = 256
N

46848

In [112]:
#hammingwindow
window = np.hamming(WINDOW_SIZE)

In [113]:
# Windowing
window_data = np.zeros(N)
for i in range(0,N-WINDOW_SIZE//2,WINDOW_SIZE//2) :
    for j in range(0,WINDOW_SIZE) : 
            window_data[i+j]+=(new_data[i+j]*window[j])/2
# frame 1 = 0 ~ 256
# frame 2 = 128 ~ 128+256
# frame 3 = 128+256 ~ 128+256+256
# frame 4 = 128+256+256 ~ 128+256+256+256
# frame 5 = 128+256+256+256 ~ 128+256+256+256+256

In [114]:
ipd.Audio(window_data,rate = sr)

In [115]:
# noise신호 뽑아내기
noise1 = window_data[0:WINDOW_SIZE]
noise2 = window_data[WINDOW_SIZE//2:WINDOW_SIZE//2+WINDOW_SIZE]
noise3 = window_data[WINDOW_SIZE//2+WINDOW_SIZE:WINDOW_SIZE//2+WINDOW_SIZE*2]
noise4 = window_data[WINDOW_SIZE//2+WINDOW_SIZE*2:WINDOW_SIZE//2+WINDOW_SIZE*3]
noise5 = window_data[WINDOW_SIZE//2+WINDOW_SIZE*3:WINDOW_SIZE//2+WINDOW_SIZE*4]
absN1 = abs(np.fft.fft(noise1))
absN2 = abs(np.fft.fft(noise2))
absN3 = abs(np.fft.fft(noise3))
absN4 = abs(np.fft.fft(noise4))
absN5 = abs(np.fft.fft(noise5))
absN = (absN1+absN2+absN3+absN4+absN5)//5

In [116]:
tmp = np.zeros(WINDOW_SIZE)
result = np.zeros(N)
theta =np.zeros(WINDOW_SIZE)
out_temp=np.zeros(N)

In [117]:
#fft,ifft
for i in range(0,N-WINDOW_SIZE//2,WINDOW_SIZE//2):
    for j in range(0,WINDOW_SIZE):
        tmp[j] = window_data[i+j]
    y = np.fft.fft(tmp)
    absY = abs(y)
    theta = np.arctan2(y.imag,y.real)
    absX = absY-absN
    X = absX*np.cos(theta)+absX*np.sin(theta)
    ifft = np.fft.ifft(X)
    for j in range(0,WINDOW_SIZE):
        out_temp[i+j] += ifft.real[j]-ifft.imag[j]
for i in range(N):
    result[i]=np.floor(out_temp[i]+0.5)

In [118]:
sr = 16000
ipd.Audio(result, rate = sr)

In [96]:
result = result.astype('int16')

In [97]:
#raw파일 저장
fi = open('조수환5.raw','w') 
result.tofile(fi) 
fi.close()